Imports - импортируем все необходимое

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [23]:
X_full = pd.read_csv('giving_departures.csv', index_col='id')

In [24]:
X_full

,down_time_train,interval_giving,idle_train,travel_time,crew_down_time,crew_down_time_plan,intermediate_down_time,route_id,sender_id,receiver_id,real_wagon_loading,real_train_weight_gross,real_train_wight_net,cargo_type_id,empty_types_id,down_time,axis_count
id,,,,,,,,,,,,,,,,,
24,0.583333,0.0,4.950000,-7.600000,2.40000,-2.050008,NaN,7506,109,3825,62,1635.0,0.0,1141,0,4.950000,248
32,2.002778,0.0,26.000000,-17.666667,NaN,NaN,NaN,7502,505,131,70,1657.9,0.0,1141,0,26.000000,280
33,2.666667,0.0,2.283333,2.966667,NaN,NaN,NaN,7503,132,131,71,1681.0,0.0,1141,0,2.283333,284
34,2.000000,0.0,6.533333,3.466667,NaN,0.900000,NaN,7503,82,131,71,1681.0,0.0,1141,0,6.533333,284
35,3.000000,0.0,3.650000,3.183333,NaN,NaN,NaN,7503,132,131,67,1583.0,0.0,1141,0,3.650000,268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5394,0.750000,0.0,1.500000,3.333333,NaN,NaN,0.433333,7492,131,505,70,6574.9,4890.5,5229,1,1.500000,280
5395,2.416667,0.0,5.733333,2.683333,NaN,NaN,NaN,7502,505,131,69,1647.0,0.0,1141,0,5.733333,276
5396,0.500000,0.0,2.350000,35.900000,5.15004,1.116672,4.266667,7500,485,82,53,4961.8,3708.0,102,1,2.350000,212


Data explorationData exploration

In [25]:
print(X_full.columns)

Index(['down_time_train', 'interval_giving', 'idle_train', 'travel_time',
       'crew_down_time', 'crew_down_time_plan', 'intermediate_down_time',
       'route_id', 'sender_id', 'receiver_id', 'real_wagon_loading',
       'real_train_weight_gross', 'real_train_wight_net', 'cargo_type_id',
       'empty_types_id', 'down_time', 'axis_count'],
      dtype='object')


In [26]:
X_full.sample(5)

,down_time_train,interval_giving,idle_train,travel_time,crew_down_time,crew_down_time_plan,intermediate_down_time,route_id,sender_id,receiver_id,real_wagon_loading,real_train_weight_gross,real_train_wight_net,cargo_type_id,empty_types_id,down_time,axis_count
id,,,,,,,,,,,,,,,,,
31597,0.466667,0.0,1.750000,2.950000,NaN,NaN,0.216667,7492,131,505,70,6552.7,4891.6,5229,1,1.750000,280
23793,5.233333,0.0,4.450000,2.816667,NaN,NaN,0.166667,7502,107,131,70,1664.0,0.0,1141,0,4.450000,280
14870,3.416667,0.0,3.716667,2.783333,NaN,NaN,0.083333,7502,505,131,68,1616.0,0.0,1141,0,3.716667,272
8424,1.666667,0.0,11.333333,3.250000,NaN,NaN,0.166667,7492,131,505,70,6565.3,4893.0,5229,1,11.333333,280
30285,0.566667,0.0,1.400000,3.300000,NaN,NaN,0.250000,7492,131,107,70,6552.6,4892.0,5229,1,1.400000,280


In [27]:
#X_full.describe(include='object')

In [28]:
X_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33246 entries, 24 to 5969
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   down_time_train          33246 non-null  float64
 1   interval_giving          33246 non-null  float64
 2   idle_train               33246 non-null  float64
 3   travel_time              33246 non-null  float64
 4   crew_down_time           11743 non-null  float64
 5   crew_down_time_plan      3834 non-null   float64
 6   intermediate_down_time   19841 non-null  float64
 7   route_id                 33246 non-null  int64  
 8   sender_id                33246 non-null  int64  
 9   receiver_id              33246 non-null  int64  
 10  real_wagon_loading       33246 non-null  int64  
 11  real_train_weight_gross  33246 non-null  float64
 12  real_train_wight_net     33246 non-null  float64
 13  cargo_type_id            33246 non-null  int64  
 14  empty_types_id           33

Data preparation - осуществляем базовые преобразования датасета

In [29]:
# Удаляем из датасета строки, в которых недостаточно данных для обучения.
X_full.dropna(axis=0, subset=['crew_down_time'], inplace=True)
# Разбиваем датасет на цель (y) и входные данные (X)
y = X_full.crew_down_time
X_full.drop(['crew_down_time'], axis=1, inplace=True)

# Разбиваем датасет на train/val
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y,
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [31]:
# В наших данных есть нечисловые значения. Они затрудняют обработку
# поэтому мы оставим только те данные, где нечисловые значения имеют меньше 10 разных значений
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and
                    X_train_full[cname].dtype == "object"]

# и все числовые данные
numerical_cols = [cname for cname in X_train_full.columns if
                X_train_full[cname].dtype in ['int64', 'float64']]

# все столбцы, в которых количество пропусков больше 10% X[cname].isnull().sum()

# оставим в датасете только те колонки, с которыми хотим работать дальше
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

Sanity check

In [32]:
print(len(X_train.columns), len(X_train_full.columns))
X_train.head()

16 16


,down_time_train,interval_giving,idle_train,travel_time,crew_down_time_plan,intermediate_down_time,route_id,sender_id,receiver_id,real_wagon_loading,real_train_weight_gross,real_train_wight_net,cargo_type_id,empty_types_id,down_time,axis_count
id,,,,,,,,,,,,,,,,
32269,0.600000,0.0,4.366667,27.783333,NaN,2.116667,7505,82,1465,69,1640.0,0.0,1141,0,4.366667,276
9070,0.583333,0.0,1.833333,31.416667,1.866672,4.200000,8590,1506,109,48,1136.5,0.0,1141,0,1.833333,192
19334,0.500000,0.0,3.050000,28.700000,NaN,3.950000,7498,131,109,64,5990.2,4469.5,5229,1,3.050000,256
31692,0.483333,0.0,6.583333,33.766667,0.716664,4.483333,7474,1465,82,53,4959.7,3705.0,102,1,6.583333,212
30277,0.516667,0.0,2.666667,31.233333,NaN,1.083333,7500,485,82,53,4961.6,3703.0,102,1,2.666667,212


#Pipeline

Baseline - базовый пайплайн, от которого мы будем отталкиваться в дальнейшем

###Data preparation (preprocessing)

In [33]:
# Инициализируем препроцессор для заполнения недостающих числовых значений
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

# Инициализируем препроцессор для заполнения недостающих НЕ числовых значений
# Добавляем шаг кодирования НЕ числовых значений в числовые
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Создаем общий препроцессор данных, соединив первые два
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
    ])

### Model

In [34]:
# Инициализируем модель
model = RandomForestRegressor(n_estimators=200, random_state=0, verbose=True)

In [35]:
# Создаем пайплайн из модели и препроцессора
classifier = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)],
                      verbose=True)

### Training

In [36]:
# Передаем обучающие данные в пайплайн, обучаем
classifier.fit(X_train, y_train)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    2.9s


[Pipeline] ............. (step 2 of 2) Processing model, total=  12.2s


[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:   12.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   12.1s finished


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['down_time_train',
                                                   'interval_giving',
                                                   'idle_train', 'travel_time',
                                                   'crew_down_time_plan',
                                                   'intermediate_down_time',
                                                   'route_id', 'sender_id',
                                                   'receiver_id',
                                                   'real_wagon_loading',
                                                   'real_train_weight_gross',
                                                   'real_train_wight_net',
                                                   'cargo_type_id',
                                                   'empty_types_id',
                                                   'down_time', 'axis_count']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model',
                 RandomForestRegressor(n_estimators=200, random_state=0,
                                       verbose=True))],
         verbose=True)

### Model evaluation

In [37]:
# Передаем валидационные данные в пайплайн, получаем предсказания
preds = classifier.predict(X_valid)

# Оцениваем точность модели
print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 1.2438348615070243


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished


In [16]:
y_valid.mean()

np.float64(7.800708020338983)

In [38]:
(1 - 1.2438348615070243/7.800708020338983)*100

84.0548465823366

In [39]:
print('MSE:', mean_squared_error(y_valid, preds))

MSE: 6.763919142067289


In [40]:
routes_to_check = 80
print(X_valid.iloc[routes_to_check].to_string())
print()
print(y_valid.iloc[routes_to_check])

down_time_train               0.583333
interval_giving               0.000000
idle_train                    0.333333
travel_time                   3.833333
crew_down_time_plan                NaN
intermediate_down_time             NaN
route_id                   7506.000000
sender_id                   109.000000
receiver_id                3825.000000
real_wagon_loading           63.000000
real_train_weight_gross    1658.000000
real_train_wight_net          0.000000
cargo_type_id              1141.000000
empty_types_id                0.000000
down_time                     0.333333
axis_count                  252.000000

1.699992


In [41]:
classifier.predict(X_valid.iloc[routes_to_check:routes_to_check + 1])

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished


array([1.530918])

In [42]:
r2_score(y_valid, preds)

0.8585033152289209